In [341]:
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [342]:
df = pd.read_csv('heart_cleveland_upload.csv')
df = df.dropna() #removing any row that might have empty cells to avoid errors


In [343]:
X = df.drop("condition",axis=1)
y = df["condition"]


In [344]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42) #splitting the 67% of dataset for training and 33% for testing


In [345]:
scaler = StandardScaler()

#normalizing the data

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [346]:
class HeartDiseasePredictionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=13,out_features=26)
    self.layer_2 = nn.Linear(in_features=26,out_features=20)
    self.layer_3 = nn.Linear(in_features=20,out_features=1)
    self.relu = nn.ReLU() #introducing relu because its a good activation function for mitigating vanishing gradients
    self.sigmoid = nn.Sigmoid() #good final layer activation function for binary classification problems
  
  def forward(self,X):
    X = self.layer_1(X)
    X = self.relu(X)
    X = self.layer_2(X)
    X = self.relu(X)
    X = self.layer_3(X)
    #X = self.sigmoid(X)
    return X.squeeze(dim=1)

model = HeartDiseasePredictionModel()

In [347]:
#converting datasets to tensors

X_train = torch.tensor(X_train,dtype=torch.float)
X_test = torch.tensor(X_test,dtype=torch.float)
y_train = torch.tensor(y_train.values,dtype=torch.float)
y_test = torch.tensor(y_test.values,dtype=torch.float)

In [348]:
#picking our loss functions and optimizer

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.0001)

In [349]:
def accuracy_fn(y_preds,y_test):
  y_preds = torch.round(torch.sigmoid(y_preds))
  correct = torch.eq(y_preds,y_test).sum().item()

  acc  = (correct / len(y_test)) * 100
  return f"{round(acc,2)}%"

In [350]:
import copy
best_test_loss = float('inf')
epochs_without_improvement = 0
patience = 3
best_model_weights = None

epochs = 5000
for epoch in range(epochs):
  model.train()
  y_preds = model(X_train)
  loss = loss_fn(y_preds,y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 200 == 0:
    model.eval()
    with torch.inference_mode():
      test_preds = model(X_test)
      test_loss = loss_fn(test_preds,y_test)

      if test_loss <= best_test_loss:
        best_test_loss = test_loss
        epochs_without_improvement = 0
        best_model_weights = copy.deepcopy(model.state_dict())
        print(accuracy_fn(test_preds,y_test))
      else:
        epochs_without_improvement += 1
      
      if epochs_without_improvement >= patience:
        break
      
      
print("Best", best_test_loss)
model.load_state_dict(best_model_weights)

56.57%
73.74%
80.81%
80.81%
80.81%
81.82%
Best tensor(0.4205)


<All keys matched successfully>

In [351]:
torch.save(model.state_dict(),'HeartDiseasePredictionModel.pth')